# Image Compression & Classificaiton Prediction

### Team 1 Final Project Report
Arthur: Lydia Niu, Ryan Kim, Stuart Arief, Colin Pham, Jonathan Chun

# Introduction

After compressing images using k-means clustering, we will turn compression into a prediction problem. We are using k-means clustering to compress images, measure how much quality is lost, and then use a supervised model (KNN) to predict which images belong to a certain category.

## Question: 
Since images vary in how compressible they are, can we measure and predict compression quality?
Can we predict which images are easier to compress?


# Dataset
[need]link insert
[need] categories + explanation of nature?

# Presentation Slides link
[need]link insert

# Technique 

# EDA - Before Compression

# Compression - K-means clustering

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt



# ello - READ THIS 

# make sure cv2 is downloaded. Change file input/output names as well as K values below  
# sometimes it might take a fat minute to load
#------------------------------------------------------------------------------------------------
image_path = "architecture/arc1.jpg"  # CHANGE THIS TO THE CORRECT FILE NAME DO NOT FORGET 
output_path = "compressed_image.jpg"  
K = 32  # MODIFY K  
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Could not load image at {image_path}")
#------------------------------------------------------------------------------------------------

# image dimensions
height, width = image.shape[:2]

# turn into 2d array of RBG values
flat_pixels = image.reshape(-1, 3).astype(np.float32)  
num_pixels = flat_pixels.shape[0]  

# choose k 
random_indices = np.random.choice(num_pixels, size=K, replace=False)
representative_colors = flat_pixels[random_indices]  
diff = flat_pixels[:, None, :] - representative_colors[None, :, :]
distances_sq = np.sum(diff**2, axis=2)  

# Distance to nearest color calculations
nearest_color_idx = np.argmin(distances_sq, axis=1)  
quantized_flat_pixels = representative_colors[nearest_color_idx]  
quantized_flat_pixels = np.rint(quantized_flat_pixels).astype(np.uint8)
quantized_image = quantized_flat_pixels.reshape(height, width, 3)

cv2.imwrite(output_path, quantized_image)
print(f"Compressed image saved as {output_path} with {K} colors.")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
quantized_image_rgb = cv2.cvtColor(quantized_image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image_rgb)
plt.title("Original Image")
plt.axis('off')  
plt.subplot(1, 2, 2)
plt.imshow(quantized_image_rgb)
plt.title(f"Quantized Image (K={K} colors)")
plt.axis('off')
plt.tight_layout()
plt.show()


[ WARN:0@427.161] global loadsave.cpp:275 findDecoder imread_('architecture/arc1.jpg'): can't open/read file: check file path/integrity


FileNotFoundError: Could not load image at architecture/arc1.jpg

# EDA - Matrix of SSIM and MSE for Image Comparison

In [1]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
from skimage.metrics import structural_similarity as ssim 

# --- Metric Calculation Function ---
def calculate_metrics(original_img, compressed_img):
    """
    Calculates MSE and SSIM between the original and compressed images.
    """
    
    # Ensure images have the same shape
    if original_img.shape != compressed_img.shape:
        raise ValueError("Original and compressed images must have the same dimensions.")
        
    # 1. Mean Squared Error (MSE)
    error = np.sum((original_img.astype("float") - compressed_img.astype("float")) ** 2)
    N = float(original_img.shape[0] * original_img.shape[1] * original_img.shape[2])
    mse = error / N
    
    # 2. Structural Similarity Index Measure (SSIM)
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    compressed_gray = cv2.cvtColor(compressed_img, cv2.COLOR_BGR2GRAY)

    # FIX: Only capture the SSIM score, as your skimage version is not returning the tuple (score, diff_map).
    ssim_score = ssim(original_gray, compressed_gray, data_range=255, channel_axis=None)
    
    return mse, ssim_score

# ---------------------------------------------------------------------------------

In [ ]:

def generate_category_matrix(category_name, prefix, k_values, originals_dir="originals", compressed_dir="compressed"):
    """
    Generates the error matrix for a single image category, sorted by image ID and K-Value.
    
    Returns:
        pandas.DataFrame: The resulting error matrix for the category.
    """
    
    original_path = os.path.join(originals_dir, category_name)
    compressed_mod_path = os.path.join(compressed_dir, f"{category_name}_mod")
    
    # Finds files (e.g., 'arc1.jpg', 'arc2.jpg')
    original_files = glob.glob(os.path.join(original_path, f"{prefix}*.jpg"))
    
    if not original_files:
        print(f"Warning: No original files found for category '{category_name}'.")
        return pd.DataFrame()
        
    results = []
    
    print(f"\n--- Processing Category: {category_name.upper()} ---")

    for og_file_path in original_files:
        base_filename = os.path.basename(og_file_path)
        image_id = os.path.splitext(base_filename)[0]
        
        og_image = cv2.imread(og_file_path)
        if og_image is None:
            print(f"Skipping {base_filename}: could not load original image.")
            continue
            
        for K in k_values:
            # Assumes compressed filename: arc1_K4.jpg, arc1_K8.jpg, etc.
            compressed_filename = f"{image_id}_K{K}.jpg" 
            compressed_file_path = os.path.join(compressed_mod_path, compressed_filename)
            
            comp_image = cv2.imread(compressed_file_path)
            
            if comp_image is None:
                print(f"  - Missing compressed file for {image_id} at K={K}. Skipping.")
                continue

            mse, ssim_score = calculate_metrics(og_image, comp_image)
            
            # Store the numeric part of the ID (e.g., 1, 2, 3) for sorting
            image_num_id = int(''.join(filter(str.isdigit, image_id)))
            
            results.append({
                'Image_ID': image_id,
                'Image_Num_ID': image_num_id, # Used for sorting (Requirement 1)
                'K_Value': K,
                'MSE': round(mse, 4),
                'SSIM': round(ssim_score, 4)
            })

    df = pd.DataFrame(results)
    
    # Requirement 1 & 2: Sort by numeric Image ID (ascending) then by K_Value (ascending)
    df = df.sort_values(by=['Image_Num_ID', 'K_Value']).drop(columns=['Image_Num_ID']).reset_index(drop=True)
    
    return df
# ---------------------------------------------------------------------------------

In [ ]:
# --- Define your configuration ---
# All K values will be checked
K_VALUES_TO_CHECK = [4, 8, 16] 

CATEGORIES = [
    {'name': 'architecture', 'prefix': 'arc'},
    {'name': 'landscapes', 'prefix': 'land'},
    {'name': 'portraits', 'prefix': 'port'},
    {'name': 'still_life', 'prefix': 'life'},
    {'name': 'urban', 'prefix': 'urb'},
]

# --- Run the generation and collect matrices ---
all_matrices = {}
overall_averages = {}

for cat in CATEGORIES:
    df_matrix = generate_category_matrix(
        category_name=cat['name'],
        prefix=cat['prefix'],
        k_values=K_VALUES_TO_CHECK
    )
    
    if not df_matrix.empty:
        # Calculate the overall average for the category (Requirement 3)
        avg_mse = df_matrix['MSE'].mean()
        avg_ssim = df_matrix['SSIM'].mean()
        overall_averages[cat['name']] = {'Avg_MSE': round(avg_mse, 4), 'Avg_SSIM': round(avg_ssim, 4)}
    
    all_matrices[cat['name']] = df_matrix

# --- Display the results and summaries ---

print("\n\n#####################################################")
print("#### DETAILED ERROR MATRIX SUMMARY BY CATEGORY ####")
print("#####################################################")

# Display the detailed matrices
for category, matrix in all_matrices.items():
    print(f"\n\n--- DETAILED MATRIX: {category.upper()} (K={K_VALUES_TO_CHECK}) ---")
    if not matrix.empty:
        print(matrix.to_markdown(index=False))
    else:
        print("No data available for this category.")

# Display the overall summary (Requirement 3)
print("\n" + "="*53)
print("### CATEGORY OVERALL PERFORMANCE SUMMARY ###")
print("="*53)

# Create a DataFrame for the summary table
summary_df = pd.DataFrame.from_dict(overall_averages, orient='index')
summary_df.index.name = "Category"

# Rename columns for clarity
summary_df.columns = ['Average MSE', 'Average SSIM']

# Display the summary table
print(summary_df.to_markdown())
print("="*53)


--- Processing Category: ARCHITECTURE ---
  - Missing compressed file for arc14 at K=16. Skipping.
  - Missing compressed file for arc14 at K=32. Skipping.
  - Missing compressed file for arc14 at K=64. Skipping.


[ WARN:0@42.234] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc14_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.235] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc14_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.236] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc14_K64.jpg'): can't open/read file: check file path/integrity


  - Missing compressed file for arc15 at K=16. Skipping.
  - Missing compressed file for arc15 at K=32. Skipping.
  - Missing compressed file for arc15 at K=64. Skipping.
  - Missing compressed file for arc17 at K=16. Skipping.
  - Missing compressed file for arc17 at K=32. Skipping.
  - Missing compressed file for arc17 at K=64. Skipping.


[ WARN:0@42.483] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc15_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.483] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc15_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.483] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc15_K64.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.608] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc17_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.608] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc17_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@42.608] global loadsave.cpp:275 findDecoder imread_('compressed/architecture_mod/arc17_K64.jpg'): can't open/read file: check file path/integrity


  - Missing compressed file for arc16 at K=16. Skipping.
  - Missing compressed file for arc16 at K=32. Skipping.
  - Missing compressed file for arc16 at K=64. Skipping.
  - Missing compressed file for arc12 at K=16. Skipping.
  - Missing compressed file for arc12 at K=32. Skipping.
  - Missing compressed file for arc12 at K=64. Skipping.


...

[ WARN:0@60.006] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb19_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.007] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb19_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.007] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb19_K64.jpg'): can't open/read file: check file path/integrity


  - Missing compressed file for urb21 at K=16. Skipping.
  - Missing compressed file for urb21 at K=32. Skipping.
  - Missing compressed file for urb21 at K=64. Skipping.
  - Missing compressed file for urb20 at K=16. Skipping.
  - Missing compressed file for urb20 at K=32. Skipping.
  - Missing compressed file for urb20 at K=64. Skipping.


[ WARN:0@60.321] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb21_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.322] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb21_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.322] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb21_K64.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.403] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb20_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.403] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb20_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.403] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb20_K64.jpg'): can't open/read file: check file path/integrity


  - Missing compressed file for urb22 at K=16. Skipping.
  - Missing compressed file for urb22 at K=32. Skipping.
  - Missing compressed file for urb22 at K=64. Skipping.
  - Missing compressed file for urb23 at K=16. Skipping.
  - Missing compressed file for urb23 at K=32. Skipping.
  - Missing compressed file for urb23 at K=64. Skipping.


#####################################################
#### FINAL ERROR MATRIX SUMMARY BY CATEGORY ####
#####################################################


--- MATRIX FOR: ARCHITECTURE ---
No data available for this category.


--- MATRIX FOR: LANDSCAPES ---
No data available for this category.


--- MATRIX FOR: PORTRAITS ---
No data available for this category.


--- MATRIX FOR: STILL_LIFE ---
No data available for this category.


--- MATRIX FOR: URBAN ---
No data available for this category.


[ WARN:0@60.547] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb22_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.547] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb22_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.548] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb22_K64.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.723] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb23_K16.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.723] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb23_K32.jpg'): can't open/read file: check file path/integrity
[ WARN:0@60.723] global loadsave.cpp:275 findDecoder imread_('compressed/urban_mod/urb23_K64.jpg'): can't open/read file: check file path/integrity


# EDA of K-means Clustering

# Technique 2: KNN (Supervised Learning)

# EDA of KNN

# Analysis

As evident by the compressed images, the colors are approximated using K-Means method and then stored into clusters based on their similarity. From this, an approximation of the colors is formed and applied to all the pixels that fall within that particular cluster. This means that the larger the defined K value is, the closer the produced image will be to the original image. 

In terms of the error, portraits have the lowest average MSE at 276.53 and a low overall SSIM of 0.7857. This indicates an overall low compression quality for this category of image. While the numerical change between a pixel's color value is low (MSE), the low SSIM means that the structural information of the entire image is poor. A possible explanation for these values would be the smoothness of the subjects' skin, subtle lighting with tonal gradients, and blurred backgrounds. All of these factors contribute to making a complex image that is difficult for the colors to be approximated and put into clusters.

On the other hand, Still Life images have the highest average SSIM at 0.8464, and an average MSE of 305.19. This indicates that it has the best preservation of quality after the compression algorithm is run. An explanation for this high performance is from the nature of Still Life images as a whole, where objects are well defined and have relatively smooth surfaces, making color clustering an effective method to use for compression.

# Conclusion

Going back to our hypothesis and cross referencing it to our matrix of results for each category, we can conclude that compression performance is indeed worse for images with higher edge density and color diversity when the k value is low. Architecture and Urban images exhibit high average MSE at k=4 (452 and 453 respectively), and low SSIM (0.796 and 0.729 respectively). This combination indicates a low compression performance due to there being an insufficient amount of clusters to accomodate the high variety of colors, ultimately resulting in a major loss in quality after compression is applied.

# Team Contribution

Ryan Kim -

Stuart Arief - Analysis

Lydia Niu - 

Jonathan Chun - 

Colin Pham - 